# The hands-on workflow

The full development workflow we will follow is:

![workflow](images/workflow.png)

In this notebook:
 - From NIR, we will export the model to **snnTorch**
 - With the snnTorch model, we can make use of **NeuroBench** to obtain metrics related to the execution of the model.

# Why NeuroBench?

When improving the performance of SNNs execution on hardware you need 3 main components:
 1. Information about your **hardware**: What are your bottlenecks? -> Done with hardware details or with **benchmarks**.
 2. Information about your **network**: What is the layer that has more of X (input spikes, memory accesses etc)
 3. A tool that uses this information to **inform** the **training** procedure and the **hardware deployment**

**NeuroBench** covers the second point. It is a hardware agnostic tool to extract relevant metrics from your model.

For example:

 - From your hardware, you know the main bottleneck is memory access (synaptic memory reads), in other words, reading the weights
 - From NeuroBench, you understand the layer that is performing the most memory accesses is layer 3
 - With this information you can:
    - Tune your training procedure to reduce the activation on layer 2
    - Prune weights in the layer 2 -> layer 3 matrix
    - During deployment, assign more cores to layer 3, so that its computation can be speed up.

# Package installation 

Execute only if you are usng Goolge Colab. If you are using your own local environment make sure you installed the dependecies first

In [1]:
! pip install snntorch
! pip install neurobench
! pip install tonic
! pip install nir==1.0.4
! pip install numpy --upgrade

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tonic 1.6.0 requires numpy<2.0.0, but you have numpy 2.3.2 which is incompatible.
numba 0.61.2 requires numpy<2.3,>=1.24, but yo

# Import packages

In [2]:
import torch
import torch.nn as nn
import snntorch as snn
import numpy as np
from snntorch.import_nir import import_from_nir
import nir

# Dataset definition

In [3]:
from tonic import datasets, transforms

def get_data_loaders(batch_size):

    transform = transforms.Compose([
        transforms.ToFrame(sensor_size=(34, 34, 2), n_time_bins=30, include_incomplete=True),
        lambda x: torch.from_numpy(x.astype(np.float32)),
    ])

    trainset = datasets.NMNIST('./data', train=True, transform=transform)
    testset = datasets.NMNIST('./data', train=False, transform=transform)

    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=0,
        drop_last=True, pin_memory=True
    )
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, num_workers=0,
        drop_last=True, pin_memory=True
    )

    return trainloader, testloader

/home/innuce-ws/miniforge3/envs/speck/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# NIR

Import model using NIR and convert the model into a torch based Neural Network using snnTorch

In [14]:
def get_nir_net (model):
    graph = nir.read(model)
    graph.nodes.keys()
    net = import_from_nir(graph)
    print(net)
    return net

Once the model has been converted in snnTorch inizialize the neurons membrane potential 

In [15]:
def init_mem_pot(net):
  modules = [e.elem for e in net.get_execution_order()]

  # init all I&F neurons
  mem_dict = {}
  for idx, module in enumerate(modules):
    if isinstance(module, snn.Leaky):
      module.mem = module.init_leaky()

# NeuroBench

Benchmarking using this tool to extract useful statistics from our model

In [16]:
from neurobench.processors.postprocessors import ChooseMaxCount
from neurobench.benchmarks import Benchmark
from neurobench.models import SNNTorchModel

from neurobench.metrics.workload import (
    ActivationSparsity,
    MembraneUpdates,
    SynapticOperations,
    ActivationSparsityByLayer,
)
from neurobench.metrics.static import (
    ParameterCount,
    Footprint,
    ConnectionSparsity,
)

def run_neuro_bench(net):
    _, testloader = get_data_loaders(batch_size=32)


    model = SNNTorchModel(net, custom_forward=False)

    static_metrics = [ParameterCount, Footprint, ConnectionSparsity]
    workload_metrics = [ActivationSparsity, ActivationSparsityByLayer,MembraneUpdates, SynapticOperations]

    benchmark = Benchmark(
        model, testloader, [], [], [static_metrics, workload_metrics]
    )
    return benchmark.run(verbose=False)

In [21]:
net_100 = get_nir_net('exported_models/100.nir')
init_mem_pot(net_100)
results_100 = run_neuro_bench(net_100)

replace rnn subgraph with nirgraph
GraphExecutor(
  (1): Conv2d(2, 8, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (10): Leaky()
  (11): Flatten(start_dim=0, end_dim=-1)
  (2): Leaky()
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (4): Leaky()
  (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (6): Leaky()
  (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (8): Leaky()
  (9): Conv2d(64, 10, kernel_size=(3, 3), stride=(np.int64(1), np.int64(1)))
  (input): Identity()
  (output): Identity()
)
Running benchmark


  0%|          | 0/312 [00:00<?, ?it/s]/home/innuce-ws/miniforge3/envs/speck/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 312/312 [02:07<00:00,  2.44it/s]


In [22]:
for key, value in results_100.items():
    print(key, value)

ParameterCount 30226
Footprint 121004
ConnectionSparsity 0.0
ActivationSparsity 0.9408151254839145
ActivationSparsityByLayer {'10': 0.9924626068376068, '2': 0.9240374189244891, '4': 0.9288202404711736, '6': 0.9781831263354701, '8': 0.9954229197271189}
MembraneUpdates 90419.66095753205
SynapticOperations {'Effective_MACs': 49208.41586538462, 'Effective_ACs': 469565.2640224359, 'Dense': 9379200.0}


In [19]:
net_3 = get_nir_net('exported_models/3.nir')
init_mem_pot(net_3)
results_3 = run_neuro_bench(net_3)

replace rnn subgraph with nirgraph
GraphExecutor(
  (1): Conv2d(2, 8, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (10): Leaky()
  (11): Flatten(start_dim=0, end_dim=-1)
  (2): Leaky()
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (4): Leaky()
  (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (6): Leaky()
  (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (8): Leaky()
  (9): Conv2d(64, 10, kernel_size=(3, 3), stride=(np.int64(1), np.int64(1)))
  (input): Identity()
  (output): Identity()
)
Running benchmark


  0%|          | 0/312 [00:00<?, ?it/s]/home/innuce-ws/miniforge3/envs/speck/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
 44%|████▍     | 138/312 [01:05<01:38,  1.77it/s]

In [23]:
for key, value in results_3.items():
    print(key, value)

ParameterCount 30226
Footprint 121004
ConnectionSparsity 0.0
ActivationSparsity 0.9937927292751302
ActivationSparsityByLayer {'10': 1.0, '2': 0.9939435897320372, '4': 0.9930222512398438, '6': 1.0, '8': 1.0}
MembraneUpdates 95410.69941907052
SynapticOperations {'Effective_MACs': 49208.41586538462, 'Effective_ACs': 35062.0016025641, 'Dense': 9379200.0}
